In [1]:
from bs4 import BeautifulSoup
import urllib, random, re, string
import cssutils
from pprint import pprint
import numpy as np


In [2]:
 
def visible(element):
    #if element.parent.name in [ 'script', '[document]', 'head', 'title','style']:
    if element.parent.name in ['[document]', 'script', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)):
        return False
    return True

In [3]:
def isAboutPage(element):
    if 'about' in element:
        return True
    return False

In [4]:
def websiteText(url):
    htmltext = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(htmltext)
    texts = soup.findAll(text=True)
    visible_texts = filter(visible, texts)
    result = filter(lambda x: x != '\n', visible_texts)
    articleResult = ""
 
    for text in result:
        articleResult += str(text.encode("utf-8"))
 
    articleResult = str(articleResult)
    #articleResult = BeautifulSoup(articleResult, convertEntities=BeautifulSoup.HTML_ENTITIES)
 
    return articleResult

In [5]:
def getAllHrefs(url):
    hrefs = []
    html_page = urllib.request.urlopen(url)
    soup = BeautifulSoup(html_page)
 
    for link in soup.findAll('a'):
        href = link.get('href')
 
        if href[0] == '/' :
            href = url + href
        hrefs.append(href)    
 
    return hrefs

# Semantic Text

In [6]:
def getAboutPages(url):
    allHrefs = getAllHrefs(url)
 
    aboutPages = filter(isAboutPage, allHrefs)
    return aboutPages
        

# Style Text

In [7]:
def getStylePages(url):
    htmltext = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(htmltext)
    cssSheets = [link["href"] for link in soup.findAll('link') if 'stylesheet' in link.get('rel', [])]
    
    return [sheet for sheet in cssSheets if ('//' not in sheet or 'https://' in sheet)] 

In [20]:
def getFonts(url, top=3):
    
    fonts = []
    cssPagesSample = getStylePages(url)
    cssPages = list(cssPagesSample)
    
    for cssPage in cssPages:
        if cssPage[0:4]=='http':
            styleSample = cssPage
        else:
            styleSample = url+cssPage
            
        cssText = urllib.request.urlopen(styleSample).read()
        soup = BeautifulSoup(cssText)
        cssTextRaw = soup.findAll(text=True)
        
        if cssTextRaw == []:
            break
        
        for styletag in str(cssTextRaw[0]).split('}'):
            if 'font-family' in styletag:
                startPos = styletag.find('font-family')
                endPos = styletag.find(';',startPos)
                fonts.append(styletag[startPos:endPos])
                
    return fonts

In [21]:
def getBackgroundColors(url, top=3):
    
    backgroundColors = []
    cssPagesSample = getStylePages(url)
    cssPages = list(cssPagesSample)
    
    #print(cssPages)
    
    for cssPage in cssPages:
        if cssPage[0:4]=='http':
            styleSample = cssPage
        else:
            styleSample = url+cssPage
            
        cssText = urllib.request.urlopen(styleSample).read()
        soup = BeautifulSoup(cssText)
        cssTextRaw = soup.findAll(text=True)
        #print(cssTextRaw )
        if cssTextRaw == []:
            break
        for styletag in str(cssTextRaw[0]).split('}'):
            if 'background-color' in styletag:
                startPos = styletag.find('background-color')
                endPos = styletag.find(';',startPos)
                backgroundColors.append(styletag[startPos:endPos])
                
    return backgroundColors

# Testing Area

In [23]:
getFonts('https://www.stackoverflow.com/')

['font-family:inherit',
 'font-family:inherit',
 'font-family:inherit',
 'font-family:inherit',
 'font-family:inherit',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif !importan',
 'font-family:Georgia,Times New Roman,Times,serif !importan',
 'font-family:Consolas,Menlo,Monaco,Lucida Console,Liberation Mono,DejaVu Sans Mono,Bitstream Vera Sans Mono,Courier New,monospace,sans-serif !importan',
 'font-family:inherit !importan',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif',
 'font-family:inherit',
 'font-family:inherit',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif',
 'font-family:Arial,"Helvetica Neue",Helvetica,sans-serif',
 'font-family:Consolas,Menlo,Monaco,Lucida Console,Liberation Mono,DejaVu Sans Mono,Bitstream Vera Sans Mono,Courier New,monospace,sans-serif',
 'font-family:Consolas,Menlo,Monaco,Lucida Console,Liberation Mono,D

In [24]:
getBackgroundColors('https://www.stackoverflow.com/')

['background-color:#848d95',
 'background-color:#242729',
 'background-color:transparent',
 'background-color:#0c0d0e',
 'background-color:#eff0f1',
 'background-color:transparent',
 'background-color:#d3d6da',
 'background-color:#FFF8E2',
 'background-color:transparent',
 'background-color:#ffecaf',
 'background-color:#EDEEEF',
 'background-color:transparent',
 'background-color:#d2d5d7',
 'background-color:#F9EBE1',
 'background-color:transparent',
 'background-color:#f1cfb6',
 'background-color:#07C',
 'background-color:transparent',
 'background-color:#005999',
 'background-color:#FFF',
 'background-color:transparent',
 'background-color:#e6e6e6',
 'background-color:#5fba7d',
 'background-color:transparent',
 'background-color:#45a163',
 'background-color:#FFF',
 'background-color:transparent',
 'background-color:#e6e6e6',
 'background-color:#FFF',
 'background-color:transparent',
 'background-color:#e6e6e6',
 'background-color:#07C',
 'background-color:transparent',
 'background-c

In [80]:
getFonts('https://www.sfu.ca/')

['font-family:"DINWebItalic"',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold"',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebMedium"',
 'font-family:DINWebMedium,sans-serif',
 'font-family:DINWeb,sans-serif',
 'font-family:"DINWebBlack",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebLight",sans-serif',
 'font-family:"DINWebLight",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBol

In [65]:
htmltext = urllib.request.urlopen('https://www.ubc.ca/').read()
soup = BeautifulSoup(htmltext)
cssSheets = [link["href"] for link in soup.findAll('link') if 'stylesheet' in link.get('rel', [])]

[sheet for sheet in cssSheets if '//' not in sheet] 

['/_assets/css/style.min.css?2']

In [67]:
getStylePages('https://www.sfu.ca/')

['/etc/designs/clf/clientlibs/pack/head-2015.styles.min.css',
 '/content/dam/sfu/main/css/sfu-main.css',
 '/content/dam/sfu/main/redesign-2015/css/main-base-2015-head.css',
 '/content/dam/sfu/main/redesign-2015/css/main-base-2015.css',
 '/content/dam/sfu/main/css/social-block.css']

In [9]:
aboutPagesSample = getAboutPages('https://www.uwaterloo.ca/')

In [31]:
getFonts('http://www.sfu.ca')

['font-family:"DINWebItalic"',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold"',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebMedium"',
 'font-family:DINWebMedium,sans-serif',
 'font-family:DINWeb,sans-serif',
 'font-family:"DINWebBlack",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebLight",sans-serif',
 'font-family:"DINWebLight",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWeb",sans-serif',
 'font-family:"DINWebBold",sans-serif',
 'font-family:"DINWebBol

In [30]:
getBackgroundColors('http://www.sfu.ca')

['background-color:#a6192e',
 'background-color:white',
 'background-color:#EEE',
 'background-color:#f2f7f7',
 'background-color:#e7f3f9',
 'background-color:#000',
 'background-color:#a6192e',
 'background-color:#e0d10a',
 'background-color:#e0d10a',
 'background-color:#e0d10a',
 'background-color:#666',
 'background-color:#333',
 'background-color:#f7c7ce',
 'background-color:#a6192e',
 'background-color:#ebebeb',
 'background-color:#d9d9d9',
 'background-color:#ebebeb',
 'background-color:#a6192e',
 'background-color:#a6192e',
 'background-color:#000',
 'background-color:#700210',
 'background-color:#75076b',
 'background-color:#003087',
 'background-color:#e45e00',
 'background-color:#d5b000',
 'background-colo',
 'background-colo',
 'background-colo',
 'background-color:#3d3935',
 'background-color:#e9eff1',
 'background-color:#fbefef',
 'background-color:#a6192e',
 'background-color:#999',
 'background-color:#f7c7ce',
 'background-color:#f7c7ce',
 'background-color:#a6192e',
 'b

In [70]:
aboutPages

NameError: name 'aboutPages' is not defined

In [10]:
for a in aboutPages:
    print('')
    print('')
    print(a)
    print('')
    
    print(websiteText(a))



https://uwaterloo.ca/about/

b'[if lt IE 7]><div id="ie6message">Your version of Internet Explorer web browser is insecure, not supported by Microsoft, and does not work with this web site. Please use one of these links to upgrade to a modern web browser: <a href="http://www.mozilla.org/firefox/">Firefox</a>, <a href="http://www.google.com/chrome">Google Chrome</a>, <a href="http://windows.microsoft.com/en-US/internet-explorer/products/ie/home">Internet Explorer</a>.</div><![endif]'b' 'b'Skip to main'b'Skip to footer'b'University of Waterloo'b'Admissions'b'About Waterloo'b'Faculties & academics'b'Offices & services'b'Support Waterloo'b'Search'b'Menu'b'This site'b'About Waterloo home'b'Who we are'b'Facts'b'History'b'Leadership and governance'b'Rankings'b'Our differentiators'b'Learning'b'Entrepreneurship'b'Co-op and experiential learning'b'Research'b'Reports'b'Accountability'b'Contact us'b'UWaterloo'b'Admissions'b'About Waterloo'b'Faculties & academics'b'Offices & services'b'Support Wa

HTTPError: HTTP Error 404: Not Found

# Reference

In [34]:
import numpy as np

x = np.array([1,1,1,2,2,2,5,25,1,1])
unique, counts = np.unique(x, return_counts=True)

print (np.asarray((unique, counts)).T)

[[ 1  5]
 [ 2  3]
 [ 5  1]
 [25  1]]


In [60]:
bgColors = getBackgroundColors('https://www.ubc.ca')
bgColors = np.array(bgColors)

['//cdn.ubc.ca/clf/7.0.5/css/ubc-clf-full-bw.min.css', '//cloud.typography.com/6804272/781004/css/fonts.css', '/_assets/css/style.min.css?2']


HTTPError: HTTP Error 404: Not Found

In [57]:
unique, counts = np.unique(bgColors, return_counts=True)

print (np.asarray((unique, counts)).T)

[]


In [46]:
unique, counts = np.unique(bgColors, return_counts=True)

freqTable = np.asarray((unique, int(counts))).T
freqTable[freqTable[:,1].argsort()]

TypeError: only size-1 arrays can be converted to Python scalars